# 1. Введение

## Вспомним как выглядит цикл решения мл-задачи

![crisp-dm](./images/crisp-dm.png)

### Предсказание оттока клиентов

Чтобы потренироваться решать задачу **оттока**, сначала поймем зачем может понадобиться такая задача.

Представим, что у нас есть банк, и клиенты хранят у нас свои деньги, понятно, что мы можем использовать их средства для операций компании. И чем больше у нас запасы, тем больший потенциал у нас есть. Хорошо, получается, нам важен каждый клиент.

Предположим, что всего у нас 100 000 клиентов и каждый хранит на своем счете **1 миллион рублей**. Получается у нас есть **10 миллиардов**. \
**Ого! И это только средств, а сколько мы можем заработать на каждом миллионе!**


![churn2](./images/churn2.png)

А что, если вдруг 10% клиентов от нас уйдет?

**Мы потеряем миллиард рублей средств, да еще и нашу прибыль сократим. \
Нужно стараться удержать наших клиентов, каждый очень ценен!**

Хорошо, мы поняли нашу задачу - это бинарная классификация на два класса **уйдет** клиент из банка или **не уйдет**.
Будем предсказывать вероятность того, что клиент оттечет, то есть число в диапазоне от 0 до 1.


Бизнес выгода ясна, также представим, что данные мы тоже придумали откуда достать.

Теперь нам нужно понять **как же уменьшить наши потери** в миллион до какого-то небольшого числа?

![churn4](./images/churn4.png)

### Составим план действий:

#### 1) Подготовка данных
    * Выборка данных
    * Разведочный анализ данных
    * Очистка данных на основе анализа
    * Генерация вспомогательных данных
    
#### 2) Моделирование
    * Выбор алгоритма	
    * Выбор мета-метрики	
    * План тестирования алгоритма
    * Обучение моделей
    * Оценка качества модели

# 2. Подготовка данных


![churn5](./images/churn5.png)

   - [x] Выборка данных
   - [ ] Разведочный анализ данных
   - [ ] Очистка данных на основе анализа
   - [ ] Генерация вспомогательных данных
   - [ ] Построение бейзлайна

Выборка данных у нас уже есть - мы будем использовать данные с сайта kaggle для такой же задачи. \
**Ссылка на соревнование:** https://www.kaggle.com/adammaus/predicting-churn-for-bank-customers

**Разведочный анализ данных:**
В рамках разведочного анализа данных, его также называют **EDA(Exploratory Data Analysis)**, мы стремимся определить и визуализировать, какие факторы способствуют оттоку клиентов.

In [1]:
#Наши стандартные библиотеки
import numpy as np
import pandas as pd

# Для визуализации
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
train.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,8160,15637427,Lu,461,Spain,Female,25,6,0.00,2,1,1,15306.29,0
1,6333,15793046,Holden,619,France,Female,35,4,90413.12,1,1,1,20555.21,0
2,8896,15658972,Foster,699,France,Female,40,8,122038.34,1,1,0,102085.35,0
3,5352,15679048,Koger,558,Germany,Male,41,2,124227.14,1,1,1,111184.67,0
4,4315,15582276,Greco,638,France,Male,34,5,133501.36,1,0,1,155643.04,0


In [4]:
test.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,4801,15679810,Chapman,690,France,Male,39,6,0.00,2,1,0,160532.88
1,2102,15778934,Napolitani,678,Spain,Female,49,8,0.00,2,0,1,98090.69
2,4487,15660646,Fanucci,528,France,Male,35,3,156687.10,1,1,0,199320.77
3,1127,15593973,Wilkie,663,Spain,Female,33,8,122528.18,1,1,0,196260.30
4,383,15568240,Ting,492,Germany,Female,30,10,77168.87,2,0,1,146700.22


![churn8](./images/churn10.png)

In [5]:
from teacher_bot.churn_prediction import ChurnPrediction

Это Бот ProductStar он будет проверять твои задания

In [6]:
BotProductStar = ChurnPrediction()

Привет! Приятно познакомиться!


In [7]:
BotProductStar.test_task(df = train)

Думаю...
Ого! Датасет, сейчас мы будем его исследовать, интересно, что в нем.
Сам я не справляюсь - нужна твоя помощь


### Задание 1. Бот тебе его расскажет - запусти ячейку ниже.

In [8]:
BotProductStar.drop_task(answer=train)

В датафрейме есть несколько колонок, в которых слишком много уникальных значений, нужно их найти и удалить.А получившийся датафрейм передать мне в параметрах.


![churn8](./images/churn11.png)

In [9]:
train.nunique()

RowNumber          7200
CustomerId         7200
Surname            2459
CreditScore         455
Geography             3
Gender                2
Age                  68
Tenure               11
Balance            4566
NumOfProducts         4
HasCrCard             2
IsActiveMember        2
EstimatedSalary    7200
Exited                2
dtype: int64

In [10]:
train = train.drop(columns=[col for col in train.columns if train[col].nunique() >= 500])


In [11]:
train.nunique()

CreditScore       455
Geography           3
Gender              2
Age                68
Tenure             11
NumOfProducts       4
HasCrCard           2
IsActiveMember      2
Exited              2
dtype: int64

In [12]:
BotProductStar.drop_task(answer=train)

В датафрейме есть несколько колонок, в которых слишком много уникальных значений, нужно их найти и удалить.А получившийся датафрейм передать мне в параметрах.


### Задание 2. Бот тебе его расскажет - запусти ячейку ниже.

In [13]:
BotProductStar.null_task('нет')

Думаю...
Ура! Второе задание позади!
Следующая кодовая фраза Science


In [30]:
# BotProductStar.null_task(answer=) - раскомментируй меня и отправь ответ в этой переменной либо 'да' либо 'нет'

   - [x] Выборка данных
   - [ ] Разведочный анализ данных
   - [x] Очистка данных на основе анализа
   - [ ] Генерация вспомогательных данных

In [14]:
train.head()

,CreditScore,Geography,Gender,Age,Tenure,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,461,Spain,Female,25,6,2,1,1,0
1,619,France,Female,35,4,1,1,1,0
2,699,France,Female,40,8,1,1,0,0
3,558,Germany,Male,41,2,1,1,1,0
4,638,France,Male,34,5,1,0,1,0


Данные выглядят, как снимок на определенный момент времени 
Возможно баланс на заданную дату, что **оставляет много вопросов:**
    * Какая это дата и какое значение она имеет?
    * Можно ли получить баланс за определенный период времени, а не за одну дату.
    * Есть клиенты, которые вышли, но все еще имеют остаток на счете! Что бы это значило? Могли они выйти из продукта, а не из банка?
    * Что значит быть активным участником и есть ли в этом разные степени? Может быть, лучше вместо этого предоставить счет транзакций, как по кредитам, так и по дебету счета?
Разбивка на продукты, купленные клиентом, может предоставить дополнительную информацию, которая увеличит список продуктов.


![churn8](./images/churn14.png)

Здесь мы приступаем к моделированию без контекста, хотя обычно наличие контекста и лучшее понимание процесса извлечения данных дает более качественные модели.

In [ ]:
labels = 'Exited', 'Retained'
sizes = [train.Exited[train['Exited']==1].count(), train.Exited[train['Exited']==0].count()]
explode = (0, 0.1)
fig1, ax1 = plt.subplots(figsize=(10, 8))
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.title("Proportion of customer churned and retained", size = 20)
plt.show()

Таким образом, около 20% клиентов ушли. Базовая модель может предполагать, что 20% клиентов уйдут.
Учитывая, что 20% - это небольшое число, нам необходимо убедиться, что выбранная модель действительно предсказывает с большой точностью эти 20%, поскольку для банка интересно идентифицировать и сохранить клиентов.

##### Анализ категориальных переменных

In [ ]:
fig, axarr = plt.subplots(2, 2, figsize=(20, 12))
sns.countplot(x='Geography', hue = 'Exited',data = train, ax=axarr[0][0])
sns.countplot(x='Gender', hue = 'Exited',data = train, ax=axarr[0][1])
sns.countplot(x='HasCrCard', hue = 'Exited',data = train, ax=axarr[1][0])
sns.countplot(x='IsActiveMember', hue = 'Exited',data = train, ax=axarr[1][1])

**Отметим следующее:**

* Большинство данных от лиц из Франции. Возможно, у банка недостаточно ресурсов для обслуживания клиентов в тех областях, где у него меньше клиентов.
* Доля сменяющих клиентов женщин также выше, чем доля клиентов-мужчин.
* Интересно, что большинство клиентов, которые ушли, - это клиенты с кредитными картами. Учитывая, что у большинства клиентов есть кредитные карты, это может оказаться простым совпадением.
* Неудивительно, что у неактивных участников больше отток. 
* Общая доля неактивных клиентов довольно высока, можно предположить, что банку потребуется программа, направленная на превращение этой группы в активных клиентов.

##### Анализ количественных переменных

In [ ]:
fig, axarr = plt.subplots(3, 2, figsize=(20, 12))
sns.boxplot(y='CreditScore',x = 'Exited', hue = 'Exited',data = train, ax=axarr[0][0])
sns.boxplot(y='Age',x = 'Exited', hue = 'Exited',data = train , ax=axarr[0][1])
sns.boxplot(y='Tenure',x = 'Exited', hue = 'Exited',data = train, ax=axarr[1][0])
sns.boxplot(y='Balance',x = 'Exited', hue = 'Exited',data = train, ax=axarr[1][1])
sns.boxplot(y='NumOfProducts',x = 'Exited', hue = 'Exited',data = train, ax=axarr[2][0])
sns.boxplot(y='EstimatedSalary',x = 'Exited', hue = 'Exited',data = train, ax=axarr[2][1])

**Отметим следующее:**
* Нет существенной разницы в распределении кредитного рейтинга между оставшимися и оттекшими клиентами.
* Старшие клиенты оттекают чаще, чем более молодые, это намекает на разницу в предпочтениях обслуживания в возрастных категориях. Банку может потребоваться пересмотреть свой целевой рынок или пересмотреть стратегию удержания клиентов между разными возрастными группами.
* Что касается срока владения, то клиенты, находящиеся на крайнем конце (мало времени проводившие с банком или много времени в банке), с большей вероятностью уйдут, чем те, которые имеют средний срок.
* Банк теряет клиентов со значительными остатками на банковских счетах, что может снизить их доступный капитал для кредитования.
* Ни продукт, ни зарплата не оказывают существенного влияния на вероятность оттока.

В целом к разведочному анализу данных можно также отнести **построение базового решения**. Важно делать на самом первом этапе, чтобы на пальцах прикидывать, а сколько можно сэкономить денег.

**Например:**
Скажем, чтобы привлечь клиента мы тратим 100 000р, а зарабатываем в год на каждом 200 000р.

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
#возьмем случайную подвыборку из train и оценим на ней сколько мы экономим
val_sample = train.sample(frac=0.2, random_state=42).reset_index(drop=True)

In [ ]:
val_sample.Exited.value_counts()

Всего видим, что ушло 295 клиентов. 
Получается мы потеряли 295 000 000р!

Есть ли что-то, что мы могли с этим сделать?

Давайте посчитаем экономию, при предскзаании алгоритмом, который случайно возвращает 0 или 1, то есть с вероятностью 0.5 говорит, что клиент оттечет и с такой же вероятностью, что клиент не оттечет.

Отсюда поймём, а имееют ли смысл вообще акции по привлечению клиентов.

In [ ]:
np.random.seed(42)
rand_prediction = np.random.randint(2, size=len(val_sample))

In [ ]:
tn, fp, fn, tp = confusion_matrix(val_sample.Exited, rand_prediction).ravel()

In [ ]:
print(f'Удержали уходящих: {tp}')
print(f'Удерживали, но они и не собирались уходить: {fp}')
print(f'Не стали удерживать и они ушли: {fn}')
print(f'Правильно не стали удерживать: {tn}')

Посчитаем сколько мы потратили всего на привлечение:
(145 + 575) * 100 000 =  72 000 000

А сколько потеряли на тех, кто ушел:
150 000 000

Количество денег ушедших из банка **ничего не делали: 295 000 000**\
Количество денег ушедших из банка **простейшей модели: 150 000 000**

**Удержано денег в банке: 145 000 000**\
**При расходах: 72 000 000**

Ого и это мы даже машинное обучение еще не начали использовать! И взяли только 20% от всей выборки.

На этом наш разведочный анализ можно закончить - мы узнали все, что нас интересует.

   - [x] Выборка данных
   - [x] Разведочный анализ данных
   - [ ] Очистка данных на основе анализа
   - [ ] Генерация вспомогательных данных
   - [ ] Предобработка данных

# Домашнее задание

##### С помощью этого метода можно делать посылки боту с предскзаниями, а он будет возвращать результат.

In [25]:
!pip install --upgrade scikit-learn

In [15]:
import joblib

model = joblib.load('/workspaces/ml_course/Project/data/my_model_v2.pkl')

In [16]:
BotProductStar.production_quality()

Я умею оценивать качество на тесте. Метрика, которую я измеряю ROC-AUC.Тут можно отправлять сделанные предсказания. Чтобы я не запутался куда какие предсказания, давай будем присылать в виде датафрейма у которого первая колонка 'RowNumber', а вторая 'predict'. Пример посылки попробуем, когда отправим submission.


Это тестовый пример посылки, вам нужно будет точно также предсказывать вероятность для каждого наблюдения и делать посылки, отправляя их боту.

In [18]:
df = pd.read_csv("/workspaces/ml_course/Project/data/customer_churn.csv")
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Age_CreditScore_interaction,Balance_EstimatedSalary_interaction,Age_Activity_interaction,Balance_Exited_interaction
0,461,Spain,Female,25,6,0.00,2,1,1,15306.29,0,11525,0.000000e+00,25,0.0
1,619,France,Female,35,4,90413.12,1,1,1,20555.21,0,21665,1.858461e+09,35,0.0
2,699,France,Female,40,8,122038.34,1,1,0,102085.35,0,27960,1.245833e+10,0,0.0
3,558,Germany,Male,41,2,124227.14,1,1,1,111184.67,0,22878,1.381215e+10,41,0.0
4,638,France,Male,34,5,133501.36,1,0,1,155643.04,0,21692,2.077856e+10,34,0.0


In [17]:
test = pd.read_csv('./data/test.csv')

In [18]:
submission = test[['RowNumber']].copy()

In [19]:
test['Balance_EstimatedSalary_interaction'] = test['Balance'] * test['EstimatedSalary']
test['Age_Activity_interaction'] = test['Age'] * test['IsActiveMember']
test['Age_CreditScore_interaction'] = test['Age'] * test['CreditScore']



In [20]:
submission['predict'] = model.predict(test)


In [21]:
submission.head()

,RowNumber,predict
0,4801,0
1,2102,0
2,4487,0
3,1127,0
4,383,0


In [30]:

baseline = [0.5]*len(test)
submission['predict'] = baseline

In [31]:
test_transformed = model.named_steps['preprocessing'].transform(test)


KeyError: 'preprocessing'

In [40]:
submission.head()

,RowNumber,predict
0,4801,0.5
1,2102,0.5
2,4487,0.5
3,1127,0.5
4,383,0.5


In [22]:
BotProductStar.production_quality(answer=submission)

Запускаю тестирование...
Проверяю метрики...
Твой результат: 0.7229374139745769
Думаю...
Нужно качество больше 0.80, я верю - у тебя получится!


Следующая часть урока полностью ложится на ваши плечи:
* Нужно правильно сгенерировать вспомогательные признаки, но сильно можно не увлекаться.
* Правильно их обработать для подачи в модель.
* И построить модель, которая будет давать достаточное качество.

Для защиты проекта необходимо будет построить модель, качество которой дает ROC-AUC не ниже 0.8. Оцениваться модель будет с помощью бота, а точнее функции - `BotProductStar.production_quality(answer=submission)`



submission - это датафрем, в котором в первой колонке указан RowNumber из файла test, а в колонке predict предсказанная вероятность ухода клиента.

Здесь можно продолжить построение модели. Давайте вспомним, какие пункты осталось сделать.

### Подготовка данных
   - [x] Выборка данных
   - [x] Разведочный анализ данных
   - [ ] Очистка данных на основе анализа
   - [ ] Генерация вспомогательных данных
   - [ ] Предобработка данных

####  Моделирование
   - [ ]  Выбор алгоритмов	
   - [ ]  Выбор мета-метрики	
   - [ ]  План тестирования алгоритма
   - [ ]  Обучение моделей
   - [ ]  Оценка качества модели